In [4]:
!sudo apt-get update
!sudo apt-get install stockfish

Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease [3632 B]
Get:3 https://dl.yarnpkg.com/debian stable InRelease [17.1 kB]                 
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]      
Get:6 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease [3961 B]
Get:7 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main all Packages [2938 B]
Get:8 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main amd64 Packages [353 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Get:10 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:12 https://dl.yarnpkg.com/debian stable/main all Packages [11.

In [5]:
!pip install --upgrade pip
!pip install chess zstandard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 43.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 82.5 MB/s eta 0:00:00
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147877 sha256=3f6779346142dc5c9defc6bacb21ac5c11af9920a6ec6268ab68dec67aa962bb
  Stored in directory: /home/codespace/.cache/pip/wheels/83/1f/4e/8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [chess]32m1/2 [chess]


Edit pgn file to remove unnecessary details

In [ ]:
import re

def reinstate_move_numbers(moves):
    tokens = moves.strip().split()
    result = []
    for i in range(0, len(tokens), 2):
        move_number = i // 2 + 1
        if i + 1 < len(tokens):
            result.append(f"{move_number}. {tokens[i]} {tokens[i+1]}")
        else:
            result.append(f"{move_number}. {tokens[i]}")
    return " ".join(result)

def clean_and_reformat_pgn(pgn_text):
    games = re.split(r'\n\s*\n', pgn_text.strip())

    cleaned_games = []

    for game in games:
        # Trenne Header und Züge
        header_lines = []
        move_lines = []

        for line in game.strip().splitlines():
            if line.startswith('['):
                header_lines.append(line)
            else:
                move_lines.append(line)

        header = "\n".join(header_lines)
        moves_raw = " ".join(move_lines)

        # Remove unnecessary symbols (information)
        moves = re.sub(r"\{[^}]*\}", "", moves_raw)
        
        moves = re.sub(r"\d+\.\.\.", "", moves)
        # Entferne alle Zugnummern (z. B. 1.)
        moves = re.sub(r"\d+\.", "", moves)
        # Reduziere Leerzeichen
        moves = re.sub(r"\s+", " ", moves).strip()
        # Setze Zugnummern korrekt neu
        moves_numbered = reinstate_move_numbers(moves)

        cleaned_game = f"{header}\n\n{moves_numbered}"
        cleaned_games.append(cleaned_game)

    return "\n\n".join(cleaned_games)

# Datei laden
with open("/workspaces/Chess_Stockfish/lichess_sample_10000 (1).pgn", "r", encoding="utf-8") as file:
    raw_pgn = file.read()

# Bereinigen und umformatieren
cleaned_pgn = clean_and_reformat_pgn(raw_pgn)

# In Datei schreiben
with open("/workspaces/Chess_Stockfish/cleaned.pgn", "w", encoding="utf-8") as out_file:
    out_file.write(cleaned_pgn)



Reduce empty lines in pgn

In [ ]:
def reduce_blank_lines_in_pgn(input_path, output_path):
    with open(input_path, "r", encoding="utf-8") as infile:
        lines = infile.readlines()

    cleaned_lines = []
    current_game = []
    blank_line_count = 0

    def flush_game(game_lines):
        """Schreibe ein einzelnes Spiel mit genau einer Leerzeile nach dem Header."""
        cleaned = []
        header_ended = False
        blank_line_count = 0
        for line in game_lines:
            stripped = line.strip()
            if stripped.startswith("["):  
                cleaned.append(line)
                header_ended = False
                blank_line_count = 0
            elif stripped == "":
                if not header_ended:
                    if blank_line_count == 0:
                        cleaned.append("\n")
                    blank_line_count += 1
                    header_ended = True
            else:
                cleaned.append(line)
                blank_line_count = 0
        cleaned.append("\n")  # <== Spiel-Ende: genau eine Leerzeile nach dem Spiel
        return cleaned

    for line in lines:
        if line.strip() == "" and not current_game:
            # Leere Zeile vor einem Spielanfang -> überspringen
            continue
        current_game.append(line)
        if line.strip().startswith("[Event") and len(current_game) > 1:
            # Neues Spiel beginnt -> vorheriges Spiel verarbeiten
            cleaned_lines.extend(flush_game(current_game[:-1]))
            current_game = [line]

    if current_game:
        cleaned_lines.extend(flush_game(current_game))

    with open(output_path, "w", encoding="utf-8") as outfile:
        outfile.writelines(cleaned_lines)
reduce_blank_lines_in_pgn("/workspaces/Chess_Stockfish/cleaned.pgn", "cleaned_one_line.pgn")

Funktion to analyse every game 

In [3]:
import chess
import chess.pgn

#Elo scales, 1978 FIDE 
def get_elo_bucket(elo):
    if elo < 1000:
        return "beginner" #Novices 
    elif elo < 1400:
        return "intermediate" #Class E and D
    elif elo < 1800:
        return "club_player" #Class C and B
    elif elo < 2200:
        return "advanced" #Class A and Experts
    else:
        return "expert" #Masters
    
def result_to_label(result, player):
    if result == "1-0":
        return "win" if player == "white" else "loss"
    elif result == "0-1":
        return "win" if player == "black" else "loss"
    elif result == "1/2-1/2":
        return "draw"
    else:
        return "unknown"    

def analyze_game(game, engine, game_nr, depth=15):
    board = game.board()
    data = []

    white_elo = int(game.headers.get("WhiteElo", 0))
    black_elo = int(game.headers.get("BlackElo", 0))
    result = game.headers.get("Result", "*")  #"1-0", "0-1", "1/2-1/2"

    for ply, move in enumerate(game.mainline_moves()):
        san_move = board.san(move)

        # Bewertung vor dem Zug
        info_before = engine.analyse(board, chess.engine.Limit(depth=depth))
        score_before = info_before["score"].white().score(mate_score=10000) if not info_before["score"].is_mate() else 10000

        # Engine-Zug berechnen
        best_move = info_before["pv"][0] if "pv" in info_before else None
        if best_move:
            best_board = board.copy()
            best_board.push(best_move)
            info_best = engine.analyse(best_board, chess.engine.Limit(depth=depth))
            score_best = info_best["score"].white().score(mate_score=10000) if not info_best["score"].is_mate() else 10000
        else:
            score_best = None

        # Tatsächlichen Zug spielen
        board.push(move)
        info_after = engine.analyse(board, chess.engine.Limit(depth=depth))
        score_after = info_after["score"].white().score(mate_score=10000) if not info_after["score"].is_mate() else 10000

        # Bewertung
        if ply % 2 == 0:
            player = "white"
            elo_level = get_elo_bucket(white_elo)
            move_quality = score_after - score_best if score_best is not None else None
        else:
            player = "black"
            elo_level = get_elo_bucket(black_elo)
            move_quality = score_best - score_after if score_best is not None else None

        total_plies = len(list(game.mainline_moves()))
        if ply < total_plies * 0.3:
            phase = 'opening'
        elif ply < total_plies * 0.7:
            phase = 'middlegame'
        else:
            phase = 'endgame'

        data.append({
            "game_nr": game_nr,
            "ply": ply,
            "move": san_move,
            "move_quality": move_quality,
            "player": player,
            "elo_level": elo_level,
            "phase": phase,
            "result": result_to_label(result, player)
        })

    return data


ModuleNotFoundError: No module named 'chess'

In [8]:
!pip install python-chess

In [10]:
#With engine2 only opening
import chess
import chess.pgn

# Elo scales, 1978 FIDE 
def get_elo_bucket(elo):
    if elo < 1000:
        return "beginner"
    elif elo < 1400:
        return "intermediate"
    elif elo < 1800:
        return "club_player"
    elif elo < 2200:
        return "advanced"
    else:
        return "expert"

def result_to_label(result, player):
    if result == "1-0":
        return "win" if player == "white" else "loss"
    elif result == "0-1":
        return "win" if player == "black" else "loss"
    elif result == "1/2-1/2":
        return "draw"
    else:
        return "unknown"

def analyze_game(game, engine, engine2, game_nr, depth=15):
    board = game.board()
    data = []

    white_elo = int(game.headers.get("WhiteElo", 0))
    black_elo = int(game.headers.get("BlackElo", 0))
    result = game.headers.get("Result", "*")

    mainline_moves = list(game.mainline_moves())
    total_plies = len(mainline_moves)
    for ply, move in enumerate(mainline_moves):
        if ply >= total_plies * 0.3:
            break

        # Lc0 evaluates position before move
        info_before = engine2.analyse(board, chess.engine.Limit(depth=depth))
        score_before = (
            info_before["score"].white().score(mate_score=10000)
            if not info_before["score"].is_mate()
            else 10000
        )

        # Stockfish calculates best move
        info_best = engine.analyse(board, chess.engine.Limit(depth=depth))
        best_move = info_best["pv"][0] if "pv" in info_best else None

        if best_move:
            best_board = board.copy()
            best_board.push(best_move)
            info_best_after = engine2.analyse(best_board, chess.engine.Limit(depth=depth))
            score_best = (
                info_best_after["score"].white().score(mate_score=10000)
                if not info_best_after["score"].is_mate()
                else 10000
            )
        else:
            score_best = None

        # Play actual move
        board.push(move)

        if ply % 2 == 0:
            player = "white"
            elo_level = get_elo_bucket(white_elo)
            move_quality = score_before - score_best if score_best is not None else None
        else:
            player = "black"
            elo_level = get_elo_bucket(black_elo)
            move_quality = score_best - score_before if score_best is not None else None

        total_plies = len(list(game.mainline_moves()))
        if ply < total_plies * 0.3:
            phase = 'opening'
        elif ply < total_plies * 0.7:
            phase = 'middlegame'
        else:
            phase = 'endgame'

        data.append({
            "game_nr": game_nr,
            "ply": ply,
            "move": san_move,
            "move_quality": move_quality,
            "player": player,
            "elo_level": elo_level,
            "phase": phase,
            "result": result_to_label(result, player),
            "eval_lc0_before": score_before
        })

    return data


In [23]:
engine = chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish")

In [24]:
import chess.engine
#engine2 = chess.engine.SimpleEngine.popen_uci("lc0_dir/lc0")

In [27]:
import json
import os
import chess
import chess.engine
import chess.pgn
from io import StringIO

DATA_FILE = "chess_data2.json"

def save_data(new_data):
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "r") as f:
            existing_data = json.load(f)
    else:
        existing_data = []

    existing_data.extend(new_data)

    with open(DATA_FILE, "w") as f:
        json.dump(existing_data, f, indent=2)

def load_data():
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "r") as f:
            return json.load(f)
    else:
        return []

# === Hauptprogramm ===
PGN_FILE = "/workspaces/Chess_Stockfish/cleaned_one_line.pgn"

engine = chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish")


MAX_GAMES = 1000
game_count = 0

# Bestehende Daten laden
all_data = load_data()

# Letzter gespeicherter game_nr ermitteln
if all_data:
    last_game_nr = max(d["game_nr"] for d in all_data)
else:
    last_game_nr = 0

with open(PGN_FILE) as pgn:
    while game_count < MAX_GAMES:
        game = chess.pgn.read_game(pgn)
        if game is None:
            break

        game_count += 1
        current_game_nr = last_game_nr + game_count
        print(f"Analysiere Spiel {current_game_nr}...")

        try:
            new_game_data = analyze_game(game, engine, game_nr=current_game_nr)

            # Erwartete Anzahl Züge (plies) im Spiel
            expected_plies = len(list(game.mainline_moves()))

            # Nur speichern, wenn das Spiel komplett analysiert wurde
            if len(new_game_data) == expected_plies:
                save_data(new_game_data)
                print(f"Spiel {current_game_nr} gespeichert.")
            else:
                print(f"Spiel {current_game_nr} NICHT gespeichert – Analyse unvollständig.")
        except KeyboardInterrupt:
            print("Analyse abgebrochen. Spiel wird nicht gespeichert.")
            break

engine.quit()


Analysiere Spiel 1...


NameError: name 'analyze_game' is not defined

In [11]:
all_data = load_data()

•Opening Theory Compliance – Do players follow book openings correctly, or do they deviate early?•Strategic Conformance – How well do players adhere to established positional and tactical guidelines?
•Endgame Fundamentals – Are winning endgame techniques executed correctly, or do deviations impact results?•Skill-Based Differences – Do higher-rated players exhibit greater conformance, and how does deviation affect performance

opening phase already analyzed avg move_quality

percentage of different elo levels beeing black and white

percentage of players range certain deviations eg difference more than 100, 100-90...: how does avg move_quality affect win/loss regardless of elo_level

compare all move_qualities of div elo levels, dif phases and whole game to percentage games won by different elo levels

Because of stockfish the later the game phase the better move_quality??

Whole game 

In [9]:
move_qualities = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None]
avg_move_quality = sum(move_qualities) / len(move_qualities)
print(f"Average move_quality whole game all elo levels: {avg_move_quality}")

move_qualities_white = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry["player"] == 'white']
avg_move_quality_white = sum(move_qualities_white) / len(move_qualities_white)
print(f"Average move_quality whole game all elo levels but only white players: {avg_move_quality_white}")

move_qualities_black = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry["player"] == 'black']
avg_move_quality_black = sum(move_qualities_black) / len(move_qualities_black)
print(f"Average move_quality whole game all elo levels but only black players: {avg_move_quality_black}")

Average move_quality whole game all elo levels: -62.39818016115266
Average move_quality whole game all elo levels but only white players: -39.97016082711085
Average move_quality whole game all elo levels but only black players: -85.16157052496575


In [12]:
total_white = len([entry["game_nr"] for entry in all_data if entry["ply"] == 0])
elo_levels = ["beginner", "intermediate", "club_player", "advanced", "expert"]
for level in elo_levels:
    number_games_elo_level_white = len([entry["game_nr"] for entry in all_data if entry["elo_level"] == level and entry["ply"] == 0])
    number_games_elo_level =  len([entry["game_nr"] for entry in all_data if entry["elo_level"] == level and entry["ply"] < 2])
    percentage_elo_level_white = number_games_elo_level_white / number_games_elo_level
    print(f"Percentage of players with elo_level {level} being player white: {percentage_elo_level_white}")    

Percentage of players with elo_level beginner being player white: 0.5037037037037037
Percentage of players with elo_level intermediate being player white: 0.5040214477211796
Percentage of players with elo_level club_player being player white: 0.4931506849315068
Percentage of players with elo_level advanced being player white: 0.5053272450532724
Percentage of players with elo_level expert being player white: 0.5185185185185185


Percentage of games won by players with different move quality in their first turn

In [13]:
thresholds = [-100, -75, -50, -25, -10]
for threshold in thresholds:
    number_games_win = len([
        entry["game_nr"] for entry in all_data
        if entry['result'] == 'win' and entry["ply"] < 2 and entry['move_quality'] > threshold
    ])
    total_games = len([
        entry["game_nr"] for entry in all_data
        if entry["ply"] < 2 and entry['move_quality'] > threshold
    ])
    percentage_games_won = number_games_win / total_games if total_games > 0 else 0
    print(f"Number of games won all elo levels with move_quality first turn over {threshold}: {number_games_win}")
    print(f"Percentage of games won all elo levels with move_quality first turn over {threshold}: {percentage_games_won}")
    print('')


Number of games won all elo levels with move_quality first turn over -100: 958
Percentage of games won all elo levels with move_quality first turn over -100: 0.4823766364551863

Number of games won all elo levels with move_quality first turn over -75: 920
Percentage of games won all elo levels with move_quality first turn over -75: 0.48092002090956615

Number of games won all elo levels with move_quality first turn over -50: 791
Percentage of games won all elo levels with move_quality first turn over -50: 0.47393648891551826

Number of games won all elo levels with move_quality first turn over -25: 566
Percentage of games won all elo levels with move_quality first turn over -25: 0.4502784407319014

Number of games won all elo levels with move_quality first turn over -10: 396
Percentage of games won all elo levels with move_quality first turn over -10: 0.47653429602888087




Average move_qualities of players with different elo levels for all phases of the game

In [14]:
elo_levels = ["beginner", "intermediate", "club_player", "advanced", "expert"]

for level in elo_levels:
    move_qualities = [ entry["move_quality"] for entry in all_data
        if entry["move_quality"] is not None and entry['elo_level'] == level
    ]
    avg_move_quality = sum(move_qualities) / len(move_qualities) if move_qualities else 0
    print(f"Average move_quality whole game {level}: {avg_move_quality}")

Average move_quality whole game beginner: -3.4766780432309443
Average move_quality whole game intermediate: -28.022786121180737
Average move_quality whole game club_player: -85.4731924741782
Average move_quality whole game advanced: -63.99712368168744
Average move_quality whole game expert: -92.48602106969206


Percentage of games won by players of different elo levels

In [15]:
elo_levels = ['beginner', 'intermediate', 'club_player', 'advanced', 'expert']
#white
for level in elo_levels:
    games = {entry["game_nr"] for entry in all_data if entry["ply"] < 1 and entry["elo_level"] == level}
    wins = {entry["game_nr"] for entry in all_data if entry["ply"] < 1 and entry["elo_level"] == level and entry['result'] == 'win'}
    
    num_games = len(games)
    num_wins = len(wins)
    percentage = num_wins / num_games if num_games > 0 else 0

    print(f"Number of games won {level}: {num_wins}")
    print(f"Percentage of games won {level}: {percentage}")
    print('')


Number of games won beginner: 31
Percentage of games won beginner: 0.45588235294117646

Number of games won intermediate: 88
Percentage of games won intermediate: 0.46808510638297873

Number of games won club_player: 181
Percentage of games won club_player: 0.5070028011204482

Number of games won advanced: 157
Percentage of games won advanced: 0.4815950920245399

Number of games won expert: 34
Percentage of games won expert: 0.4857142857142857



Phase Opening

In [16]:
move_qualities_opening = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry["phase"] == 'opening']
avg_move_quality_opening = sum(move_qualities_opening) / len(move_qualities_opening)
print(f"Average move_quality opening all elo levels: {avg_move_quality_opening}")

Average move_quality opening all elo levels: -46.518992120039634


In [17]:
elo_levels = ['beginner', 'intermediate', 'club_player', 'advanced', 'expert']

for level in elo_levels:
    move_qualities_opening = [
        entry["move_quality"]
        for entry in all_data
        if entry["move_quality"] is not None and
           entry["elo_level"] == level and
           entry["phase"] == "opening"
    ]

    if move_qualities_opening:  # schützt vor Division durch 0
        avg_move_quality = sum(move_qualities_opening) / len(move_qualities_opening)
        print(f"Average move_quality opening {level}: {avg_move_quality}")
    else:
        print(f"No data for opening phase in elo level {level}.")

Average move_quality opening beginner: -55.858823529411765
Average move_quality opening intermediate: -44.457758412006775
Average move_quality opening club_player: -45.939876957494405
Average move_quality opening advanced: -47.36063708759954
Average move_quality opening expert: -42.590250329380765


Phase Middlegame

In [18]:
move_qualities_middle = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry["phase"] == 'middlegame']
avg_move_quality_middle = sum(move_qualities_middle) / len(move_qualities_middle)
print(f"Average move_quality middlegame all elo levels: {avg_move_quality_middle}")

Average move_quality middlegame all elo levels: -94.85671976017626


In [19]:
elo_levels = ['beginner', 'intermediate', 'club_player', 'advanced', 'expert']

for level in elo_levels:
    move_qualities_middlegame = [
        entry["move_quality"]
        for entry in all_data
        if entry["move_quality"] is not None and
           entry["elo_level"] == level and
           entry["phase"] == "middlegame"
    ]

    if move_qualities_middlegame:  # schützt vor Division durch 0
        avg_move_quality = sum(move_qualities_middlegame) / len(move_qualities_middlegame)
        print(f"Average move_quality middlegame {level}: {avg_move_quality}")
    else:
        print(f"No data for middlegame phase in elo level {level}.")

Average move_quality middlegame beginner: -107.48307515777395
Average move_quality middlegame intermediate: -113.97585698488759
Average move_quality middlegame club_player: -88.68519508284339
Average move_quality middlegame advanced: -87.92431491953023
Average move_quality middlegame expert: -92.68937468225724


Endgame

In [20]:
move_qualities_endgame = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry["phase"] == 'endgame']
avg_move_quality_endgame = sum(move_qualities_endgame) / len(move_qualities_endgame)
print(f"Average move_quality endgame all elo levels: {avg_move_quality_endgame}")

Average move_quality endgame all elo levels: -34.6334765644293


In [25]:
elo_levels = ['beginner', 'intermediate', 'club_player', 'advanced', 'expert']

for level in elo_levels:
    move_qualities_endgame = [
        entry["move_quality"]
        for entry in all_data
        if entry["move_quality"] is not None and
           entry["elo_level"] == level and
           entry["phase"] == "endgame"
    ]

    if move_qualities_endgame:  # schützt vor Division durch 0
        avg_move_quality = sum(move_qualities_endgame) / len(move_qualities_endgame)
        print(f"Average move_quality endgame {level}: {avg_move_quality}")
    else:
        print(f"No data for endgame phase in elo level {level}.")

Average move_quality endgame beginner: 191.97445820433435
Average move_quality endgame intermediate: 106.8949494949495
Average move_quality endgame club_player: -122.4925285672429
Average move_quality endgame advanced: -48.65823161655255
Average move_quality endgame expert: -144.41006202618883


Percentages of games won with different ranges of avg move_quality

In [24]:
number = [-10000,-200, -150, -100, -50, 0]
number2  = [-200, -150, -100, -50, 0, 10000]

count = 0
count2 = 0
for n in range(6):
    for game_nr in range(10001):
        if game_nr == 0:
            continue
        move_qualities_dif_game_nr = [
        entry["move_quality"]
        for entry in all_data
        if entry["game_nr"] == game_nr
        and entry["player"] == 'white'
        and entry['result'] == 'win'
        ]
        #avg_move_quality_game_nr = None
        if move_qualities_dif_game_nr:
            avg_move_quality_game_nr = sum(move_qualities_dif_game_nr) / len(move_qualities_dif_game_nr)
        if avg_move_quality_game_nr <= number2[n] and avg_move_quality_game_nr > number[n]:
           count += 1

        move_qualities_dif_game_nr = [
        entry["move_quality"]
        for entry in all_data
        if entry["game_nr"] == game_nr
        and entry["player"] == 'white'
        ]
        #avg_move_quality_game_nr = None
        if move_qualities_dif_game_nr:
            avg_move_quality_game_nr = sum(move_qualities_dif_game_nr) / len(move_qualities_dif_game_nr)
        if avg_move_quality_game_nr <= number2[n] and avg_move_quality_game_nr > number[n]:
           count2 += 1
    percentage = count/count2 if count2 > 0 else 0
    print(f"Percentage of games won with average move_quality - {number[n]} < move_quality =< {number2[n]}: {percentage} ")



Percentage of games won with average move_quality - -10000 < move_quality =< -200: 0.9310344827586207 
Percentage of games won with average move_quality - -200 < move_quality =< -150: 0.9298245614035088 
Percentage of games won with average move_quality - -150 < move_quality =< -100: 0.9915730337078652 
Percentage of games won with average move_quality - -100 < move_quality =< -50: 1.0082236842105263 
Percentage of games won with average move_quality - -50 < move_quality =< 0: 1.0002042483660132 
Percentage of games won with average move_quality - 0 < move_quality =< 10000: 1.0 


In [ ]:
import chess
import chess.pgn

#Elo scales, 1978 FIDE 
def get_elo_bucket(elo):
    if elo < 1000:
        return "beginner" #Novices 
    elif elo < 1400:
        return "intermediate" #Class E and D
    elif elo < 1800:
        return "club_player" #Class C and B
    elif elo < 2200:
        return "advanced" #Class A and Experts
    else:
        return "expert" #Masters
    
def result_to_label(result, player):
    if result == "1-0":
        return "win" if player == "white" else "loss"
    elif result == "0-1":
        return "win" if player == "black" else "loss"
    elif result == "1/2-1/2":
        return "draw"
    else:
        return "unknown"    

def analyze_game_opening(game, engine, game_nr, depth=25):
    board = game.board()
    data = []
    if ply 

    white_elo = int(game.headers.get("WhiteElo", 0))
    black_elo = int(game.headers.get("BlackElo", 0))
    result = game.headers.get("Result", "*")  #"1-0", "0-1", "1/2-1/2"

    for ply, move in enumerate(game.mainline_moves()):
        san_move = board.san(move)

        # Bewertung vor dem Zug
        info_before = engine.analyse(board, chess.engine.Limit(depth=depth))
        score_before = info_before["score"].white().score(mate_score=10000) if not info_before["score"].is_mate() else 10000

        # Engine-Zug berechnen
        best_move = info_before["pv"][0] if "pv" in info_before else None
        if best_move:
            best_board = board.copy()
            best_board.push(best_move)
            info_best = engine.analyse(best_board, chess.engine.Limit(depth=depth))
            score_best = info_best["score"].white().score(mate_score=10000) if not info_best["score"].is_mate() else 10000
        else:
            score_best = None

        # Tatsächlichen Zug spielen
        board.push(move)
        info_after = engine.analyse(board, chess.engine.Limit(depth=depth))
        score_after = info_after["score"].white().score(mate_score=10000) if not info_after["score"].is_mate() else 10000

        # Bewertung
        if ply % 2 == 0:
            player = "white"
            elo_level = get_elo_bucket(white_elo)
            move_quality = score_after - score_best if score_best is not None else None
        else:
            player = "black"
            elo_level = get_elo_bucket(black_elo)
            move_quality = score_best - score_after if score_best is not None else None

        total_plies = len(list(game.mainline_moves()))
        if ply < total_plies * 0.3:
            phase = 'opening'
        elif ply < total_plies * 0.7:
            phase = 'middlegame'
        else:
            phase = 'endgame'

        data.append({
            "game_nr": game_nr,
            "ply": ply,
            "move": san_move,
            "move_quality": move_quality,
            "player": player,
            "elo_level": elo_level,
            "phase": phase,
            "result": result_to_label(result, player)
        })

    return data
